This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/policy_search.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import Integrator  # wrap_to,
from pydrake.all import (
    AutoDiffXd,
    BasicVector_,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    LeafSystem_,
    LogVectorOutput,
    MeshcatVisualizer,
    Rgba,
    SceneGraph,
    Simulator,
    Simulator_,
    StartMeshcat,
    TemplateSystem,
)
from pydrake.examples import PendulumGeometry, PendulumPlant

from underactuated import running_as_notebook
from underactuated.utils import running_as_test

if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# Policy Optimization for LQR

# Trajectory-based (stochastic) gradient descent

Note: This code is super slow right now, but will be optimized soon!

## Pendulum swing-up


In [ ]:
@TemplateSystem.define("ParameterizedController_")
def ParameterizedController_(T):
    class Impl(LeafSystem_[T]):
        def _construct(self, converter=None):
            LeafSystem_[T].__init__(self, converter)
            self.DeclareVectorInputPort("state", BasicVector_[T](2))
            self.DeclareNumericParameter(BasicVector_[T](np.zeros((9, 1))))
            self.DeclareVectorOutputPort(
                "command", BasicVector_[T](1), self.CommandOutput
            )

        def _construct_copy(self, other, converter=None):
            Impl._construct(self, converter=converter)

        def CommandOutput(self, context, output):
            x = self.get_input_port().Eval(context)
            q = x[0]
            qd = x[1]
            basis = np.array(
                [
                    1,
                    np.sin(q),
                    np.cos(q),
                    qd,
                    np.sin(q) ** 2,
                    np.sin(q) * np.cos(q),
                    qd * np.sin(q),
                    qd * np.cos(q),
                    qd**2,
                ]
            )
            alpha = context.get_numeric_parameter(0).CopyToVector()
            output[0] = alpha.dot(basis)

    return Impl


ParameterizedController = ParameterizedController_[None]  # Default instantiation


def wrap_to(value, low, high):
    range = high - low
    return value - range * np.floor((value - low) / range)


@TemplateSystem.define("RunningCost_")
def RunningCost_(T):
    class Impl(LeafSystem_[T]):
        def _construct(self, converter=None):
            LeafSystem_[T].__init__(self, converter)
            self.DeclareVectorInputPort("state", BasicVector_[T](2))
            self.DeclareVectorInputPort("command", BasicVector_[T](1))
            self.DeclareVectorOutputPort("cost", BasicVector_[T](1), self.CostOutput)

        def _construct_copy(self, other, converter=None):
            Impl._construct(self, converter=converter)

        def CostOutput(self, context, output):
            x = self.get_input_port(0).Eval(context)
            x[0] = wrap_to(x[0] - np.pi, -np.pi, np.pi)
            u = self.get_input_port(1).Eval(context)[0]
            Q = np.diag([10, 1])
            R = 1
            output[0] = x.dot(Q.dot(x)) + R * u**2

    return Impl


RunningCost = RunningCost_[None]  # Default instantiation


def simulate(alpha, N=10, set_planar=True):
    builder = DiagramBuilder()
    plant = builder.AddSystem(PendulumPlant())
    pi = builder.AddSystem(ParameterizedController())
    builder.Connect(plant.get_state_output_port(), pi.get_input_port())
    builder.Connect(pi.get_output_port(), plant.get_input_port())
    scene_graph = builder.AddSystem(SceneGraph())
    PendulumGeometry.AddToBuilder(builder, plant.get_state_output_port(), scene_graph)
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Delete()

    diagram = builder.Build()
    rng = np.random.default_rng(123)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(0 if running_as_test else 1.0)
    context = simulator.get_mutable_context()
    pi_context = pi.GetMyContextFromRoot(context)
    pi_context.get_mutable_numeric_parameter(0).SetFromVector(alpha)

    for n in range(1 if running_as_test else N):
        context.SetTime(0.0)
        context.SetContinuousState(rng.standard_normal((2, 1)))
        simulator.Initialize()
        simulator.AdvanceTo(0.1 if running_as_test else 2.0)


def plot_rollouts(alpha, N=10):
    builder = DiagramBuilder()
    plant = builder.AddSystem(PendulumPlant())
    pi = builder.AddSystem(ParameterizedController())
    builder.Connect(plant.get_state_output_port(), pi.get_input_port())
    builder.Connect(pi.get_output_port(), plant.get_input_port())
    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    diagram = builder.Build()
    rng = np.random.default_rng(123)

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    pi_context = pi.GetMyContextFromRoot(context)
    pi_context.get_mutable_numeric_parameter(0).SetFromVector(alpha)

    meshcat.Delete()
    tf = 0.1 if running_as_test else 2.5
    vertices = np.vstack([[0, tf], [0, 0], np.array([1, 1]) * np.pi])
    meshcat.SetLine("goal", vertices, rgba=Rgba(0, 0.5, 0))
    vertices = np.vstack([[0, tf], [0, 0], -np.array([1, 1]) * np.pi])
    meshcat.SetLine("goal_wrapped", vertices, rgba=Rgba(0, 0.5, 0))
    for n in range(1 if running_as_test else N):
        context.SetTime(0.0)
        context.SetContinuousState(rng.standard_normal((2, 1)))
        simulator.Initialize()
        simulator.AdvanceTo(tf)
        log = logger.FindLog(context)
        times = log.sample_times()
        vertices = np.vstack([times, 0 * times, log.data()[0, :]])
        meshcat.SetLine(f"traj{n}", vertices, rgba=Rgba(0, 0, 0.5))


def trajectory_gradient_descent(alpha0):
    builder = DiagramBuilder()
    plant = builder.AddSystem(PendulumPlant())
    plant.set_name("plant")
    pi = builder.AddSystem(ParameterizedController())
    pi.set_name("pi")
    builder.Connect(plant.get_state_output_port(), pi.get_input_port())
    builder.Connect(pi.get_output_port(), plant.get_input_port())

    running_cost = builder.AddSystem(RunningCost())
    running_cost.set_name("running_cost")
    builder.Connect(plant.get_state_output_port(), running_cost.get_input_port(0))
    builder.Connect(pi.get_output_port(), running_cost.get_input_port(1))
    cost = builder.AddSystem(Integrator(1))
    cost.set_name("integrator")
    builder.Connect(running_cost.get_output_port(), cost.get_input_port())

    diagram = builder.Build()
    diagram.set_name("trajectory gradient descent")
    # Useful if you want to poke around:
    # display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg()))

    diagram_ad = diagram.ToAutoDiffXd()
    plant_ad = diagram_ad.GetSubsystemByName("plant")
    pi_ad = diagram_ad.GetSubsystemByName("pi")
    cost_ad = diagram_ad.GetSubsystemByName("integrator")

    simulator = Simulator_[AutoDiffXd](diagram_ad)
    integrator = simulator.get_mutable_integrator()
    integrator.set_fixed_step_mode(True)
    integrator.set_maximum_step_size(0.05)

    rng = np.random.default_rng(123)

    context = simulator.get_mutable_context()
    plant_context = plant_ad.GetMyContextFromRoot(context)
    pi_context = pi_ad.GetMyContextFromRoot(context)
    cost_context = cost_ad.GetMyContextFromRoot(context)

    alpha = alpha0
    N = 3 if running_as_test else 50
    x0 = rng.standard_normal((2, N))
    learning_rate = 2e-5
    for epoch in range(2 if running_as_test else 2000):
        J = 0
        pi_context.get_mutable_numeric_parameter(0).SetFromVector(
            InitializeAutoDiff(alpha)
        )
        for n in range(N):  # TODO(russt): Do multi-threading here.
            context.SetTime(0.0)
            plant_context.SetContinuousState(x0[:, n])
            cost_context.SetContinuousState([0])
            simulator.Initialize()
            simulator.AdvanceTo(2.5)
            J += cost_context.get_continuous_state_vector().CopyToVector() / N

        dJdalpha = ExtractGradient(J)
        # TODO(russt): Use Adam here instead of SGD
        alpha -= learning_rate * dJdalpha.T

        clear_output(wait=True)
        print(f"epoch {epoch}: J = {ExtractValue(J)}")
        if epoch % 100 == 99:
            simulate(alpha, N=5, set_planar=False)
            print(alpha.T)
        if epoch % 10 == 9:
            plot_rollouts(alpha, N=5)

    return alpha


meshcat.Set2dRenderMode()
# alpha=0.01*np.ones((9,1))
# From a previous run:
alpha = np.array(
    [
        0.71027108,
        2.61142863,
        0.6379523,
        -0.23501137,
        0.64446781,
        -1.45158088,
        0.37477416,
        1.05945338,
        -0.04337835,
    ]
).reshape((9, 1))
alpha = trajectory_gradient_descent(alpha)
print(alpha.T)

In [ ]:
plot_rollouts(alpha, N=50)

In [ ]:
simulate(alpha, N=10)

# Static Output Feedback Counter-example

In [ ]:
# plt.rcParams['text.usetex'] = True


def static_output_feedback():
    A = np.array([[0, 0, 2], [1, 0, 0], [0, 1, 0]])
    B = np.array([[1], [0], [0]])
    C = np.array([[1, 1, 3]])

    k = np.linspace(0, 3, 101)
    lambda_max_real = 0 * k
    for i in range(101):
        A_cl = A - B @ (k[i] * C)
        lambda_max_real[i] = np.amax(np.real(np.linalg.eigvals(A_cl)))

    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(k, lambda_max_real)
    ax.plot(k, 0 * k, "k")
    ax.set_xlabel("$k$")
    ax.set_ylabel("$\Re(\lambda_{max})$")
    display(mpld3.display())


static_output_feedback()